In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from selenium import webdriver
import time
import numpy as np
import ast
from datetime import datetime
from multiprocessing import Pool

In [3]:
df = pd.read_csv('../data/car_links.csv')
df.head()

,car_links
0,https://www.hasznaltauto.hu/szemelyauto/volksw...
1,https://www.hasznaltauto.hu/szemelyauto/citroe...
2,https://www.hasznaltauto.hu/szemelyauto/merced...
3,https://www.hasznaltauto.hu/szemelyauto/dacia/...
4,https://www.hasznaltauto.hu/szemelyauto/merced...


In [8]:
df["collected_at"] = '2023-06-26 12:00:00'

In [10]:
df['car_links'] = df['car_links'].str.replace("#sid.*", "")

/var/folders/6p/rn1y8jlj4q5fp4y6jrn3qslc0000gn/T/ipykernel_15320/3596822670.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['car_links'] = df['car_links'].str.replace("#sid.*", "")


In [16]:
df.to_csv('../data/car_links.csv', index=False)

In [7]:
str(datetime.today())[:19]

'2023-07-02 13:22:46'

In [5]:
BASE_URL = 'https://www.hasznaltauto.hu/talalatilista/PCOG2VG3R3RDADH5S56ADN4Z3EEY7O2CQO2EEI5NGSXZLINGQROIZEVUDJAPZ6Z2NVQZVCVHLLDY47L4OJJJB43XPEGXEOUTIJB4JCV4QIGTMYWJ3BKIU4CBRPTAY5NECZNEQAJGF7OPJ6CB7EJCHBQGJIEBPMO4ZJE57JDB3CAC7LDSVHEUICRRZ3APOMHTEIC5NPWIZZWZ4JOSWHDXZHXKYDCOECG62ZWR6KNG5X3U3CQKHS4J2AVDYKJYEWO2CQGLSJZZT24ULD7GQCI3F5FE7W7EXQP4FCUIQMIW64YWGWLIAXDQRRZ3Z2F646IDCZVF2UUPJUP5QYHBEF4F7FVX6EMGUE2DEIYGPHUTR3FK74JF7QHH575DDQF7IQIW76QW7VAOIOTNZZ7B5TEX3KISPM5DHHV4VNBSJ62XQX4OTSKSKTSNMZMXPZWU67NNG6MVJLEFBIV57HKRVZNCSE6RO4TZRHAFV6QVA7MLEQOJK2GMAPQZFKCOOU4KEGPZAQNRMHTCF3GAGHAIUNK4GF3B7IRL5VR5HM3UM3OH43FVFGP3TBL43LQHO6DOVUIXRSARGRNRAT4SFL5KWN3LXNUS2MXJ5JLY23C7GFGPXN4JDYSFC67YRZMNAO7LGA52BJ663Y4YLCPMXXYFO5QBXEJO3CWBM7UWVQGQNTCQL2BFKNDIF6SBG2NOXK3AFJ4XNQQ4FUGDLRGCCXBUXNOA74OXOSO3IV67THQRSRMDHITEXBACWYDLEQ3JQ2U5TNCORXQIR5UUMZT53WUXS4OHTUPEZ3GTBR2KMA4I3CBW4TUEBHVKEH7NN7KJ7DUQ2UNWXQ3MUZVLUZPQCT2DO7DFB3GJK5QOZAHHVUJZTLP5Q53LKRJXEB57WD3X72QGI/page'


'https://www.hasznaltauto.hu/talalatilista/PCOG2VG3R3RDADH5S56ADN4Z3EEY7O2CQO2EEI5NGSXZLINGQROIZEVUDJAPZ6Z2NVQZVCVHLLDY47L4OJJJB43XPEGXEOUTIJB4JCV4QIGTMYWJ3BKIU4CBRPTAY5NECZNEQAJGF7OPJ6CB7EJCHBQGJIEBPMO4ZJE57JDB3CAC7LDSVHEUICRRZ3APOMHTEIC5NPWIZZWZ4JOSWHDXZHXKYDCOECG62ZWR6KNG5X3U3CQKHS4J2AVDYKJYEWO2CQGLSJZZT24ULD7GQCI3F5FE7W7EXQP4FCUIQMIW64YWGWLIAXDQRRZ3Z2F646IDCZVF2UUPJUP5QYHBEF4F7FVX6EMGUE2DEIYGPHUTR3FK74JF7QHH575DDQF7IQIW76QW7VAOIOTNZZ7B5TEX3KISPM5DHHV4VNBSJ62XQX4OTSKSKTSNMZMXPZWU67NNG6MVJLEFBIV57HKRVZNCSE6RO4TZRHAFV6QVA7MLEQOJK2GMAPQZFKCOOU4KEGPZAQNRMHTCF3GAGHAIUNK4GF3B7IRL5VR5HM3UM3OH43FVFGP3TBL43LQHO6DOVUIXRSARGRNRAT4SFL5KWN3LXNUS2MXJ5JLY23C7GFGPXN4JDYSFC67YRZMNAO7LGA52BJ663Y4YLCPMXXYFO5QBXEJO3CWBM7UWVQGQNTCQL2BFKNDIF6SBG2NOXK3AFJ4XNQQ4FUGDLRGCCXBUXNOA74OXOSO3IV67THQRSRMDHITEXBACWYDLEQ3JQ2U5TNCORXQIR5UUMZT53WUXS4OHTUPEZ3GTBR2KMA4I3CBW4TUEBHVKEH7NN7KJ7DUQ2UNWXQ3MUZVLUZPQCT2DO7DFB3GJK5QOZAHHVUJZTLP5Q53LKRJXEB57WD3X72QGI/page'

In [6]:
def get_links(page_number):
    url = BASE_URL+str(page_number)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    matches = soup.find_all('a', {'class':''})
    
    # Extract the hrefs from matches
    hrefs = pd.Series([match['href'] for match in matches if match.has_attr('href')])
    
    # Keep only those that a like for an car
    hrefs = hrefs[hrefs.str.contains('www.hasznaltauto.hu/szemelyauto', regex=False)].to_list()
    
    return hrefs


In [7]:
# Scrape all the links for cars from hasznaltauto.hu
# all_hrefs = [get_links(x) for x in range(1,296)]


In [5]:
"""
df_hrefs = pd.DataFrame({'car_links':np.concatenate(all_hrefs)})
df_hrefs['car_links'] = df_hrefs['car_links'].str.replace('#sid.*', '')
df_hrefs.drop_duplicates(inplace=True)
df_hrefs.reset_index(drop=True, inplace=True)
df_hrefs.to_csv('car_links.csv', index=False)
"""
df_hrefs = pd.read_csv('car_links.csv')
df_hrefs.head()#"""

,car_links
0,https://www.hasznaltauto.hu/szemelyauto/volksw...
1,https://www.hasznaltauto.hu/szemelyauto/citroe...
2,https://www.hasznaltauto.hu/szemelyauto/merced...
3,https://www.hasznaltauto.hu/szemelyauto/merced...
4,https://www.hasznaltauto.hu/szemelyauto/merced...


In [18]:
def scrape_car_data(link):
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')

        header = soup.find_all('div', {'class':'data-relay-banner'})
        
        if len(header) == 0:
            return None, None
        
        header = ast.literal_eval(header[0]['data-vehicle-information-banner-parameters'])
        advertisement_data =  soup.find_all('table', {'class':'hirdetesadatok'})
        # Get the text from xml
        advertisement_data = advertisement_data[0].get_text(separator='#')

        # Clean it
        advertisement_data = advertisement_data.strip()
        advertisement_data = re.sub(r'#\n', '', advertisement_data)

        # Get lines
        advertisement_data = advertisement_data.split('#')

        # Clean lines and create pairs
        advertisement_data = pd.Series(advertisement_data)

        # Add header to advertisment data
        advertisement_data.update(header)

        keys_id = advertisement_data.str.contains(':$')
        values_id = np.where(keys_id)[0] + 1
        keys = advertisement_data[keys_id]
        values = advertisement_data[values_id]
        advertisement_data = dict(zip(keys, values))


        # Get equipment info
        equipments = soup.find_all('div', {'class':'row felszereltseg'})
        if equipments:
            equipments = equipments[0].text.split('\n')

        # Get other info
        other = soup.find_all('div', {'class':'egyebinformacio'})
        if other:
            other = other[0].text.split('\n')
        
        # Get description 
        description = soup.find_all('div', {'class':'leiras'})
        if description:
            description = [description[0].text]        


        # Get oll special info about the car and clean it
        special_car_info = equipments + other + description
        if special_car_info:
            special_car_info = pd.Series(special_car_info)
            special_car_info = special_car_info.str.strip()
            special_car_info = special_car_info.str.lower()
            special_car_info = special_car_info.dropna()
            special_car_info = special_car_info[~special_car_info.isin(['', '-'])]

        return advertisement_data, special_car_info
    except Exception as e:
        print(e)
        return None, None

In [45]:
# Open the Pickle file in binary mode for reading
with open('../data/car_data.pickle', 'rb') as file:
    # Load the Pickle file
    res_list = pickle.load(file)


In [46]:
len(res_list)

35386

In [47]:
all_links = pd.Series([x[2] for x in res_list])


In [48]:
all_links = all_links.str.replace("#sid.*", "")

/var/folders/6p/rn1y8jlj4q5fp4y6jrn3qslc0000gn/T/ipykernel_15320/1019308625.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  all_links = all_links.str.replace("#sid.*", "")


In [49]:
len(all_links)

35386

In [50]:
is_link_duplicated = all_links.duplicated()

In [51]:
is_link_duplicated.sum()

0

In [48]:
# Update dataset with new cars
def load_existing_links(filename):
    df = pd.read_csv(filename)
    return df['car_links'].values

def get_all_links():
    all_links = []
    page_number = 1
    while True:
        links_on_page = get_links(page_number)
        if not links_on_page:
            break
        page_number += 1
        all_links.append(links_on_page)
        
    all_links =  pd.Series(np.concatenate(all_links))
    all_links = all_links.str.replace('#sid.*', '')
    return all_links


def get_new_links(existing_links):
    all_hrefs = get_all_links()

    # Extract the hrefs from page results
    new_links = all_hrefs[~all_hrefs.isin(existing_links)]
    new_links = new_links.drop_duplicates()
    
    return new_links


def update_links(filename):
    existing_links = load_existing_links(filename)
    new_links = get_new_links(existing_links)

    # Add new links to existing ones and save to csv
    all_links = list(existing_links) + list(new_links)
    df_hrefs = pd.DataFrame({'car_links': all_links})
    df_hrefs.drop_duplicates(inplace=True)
    df_hrefs.reset_index(drop=True, inplace=True)
    df_hrefs.to_csv(filename, index=False)

    return new_links

# Use the function like this:
new_links = update_links('car_links.csv')


KeyboardInterrupt: 

In [72]:
new_links = new_links[~new_links.isin(all_links)]

In [82]:
%%time
for i, link in enumerate(new_links):
    advertisement_data, special_car_info = scrape_car_data(link)
    res_list.append([advertisement_data, special_car_info, link])
    
    if (i % 100) == 0:
        print(i)
        with open('hasnaltauto_scraped.pickle', 'wb') as file:
            pickle.dump(res_list, file)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
CPU times: user 4min 33s, sys: 42.4 s, total: 5min 15s
Wall time: 21min 18s


In [66]:
i

1439